In [1]:
!pip install -r requirements.txt

In [5]:
import os
import glob
import json
import chromadb

from langchain_community.document_loaders import TextLoader
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.docstore.document import Document

%reload_ext dotenv 
%dotenv 

%run YandexLLM.ipynb
%run YandexEmbeddings.ipynb

api_key = os.getenv('YAGPT_API_KEY')
folder_id = os.getenv('YA_FOLDER_ID')

### Создание экяземпляров моделей и формирование эмбедингов документов

In [6]:
embeding_model = YandexEmbeddings(api_key, folder_id)
llm_model = YandexLLM(api_key=api_key, folder_id=folder_id)

In [7]:
doc_files = glob.glob('/home/ashneider/mount/simple_rag_yandexgpt/documents/*.txt')

doc_texts = []
for file_name in doc_files:
    data = open(file_name, "r").read()
    doc_texts.append(data)

doc_embeddings = []
for doc in doc_texts:
    doc_embeddings.append(json.loads(embeding_model.embed_documents(doc))["embedding"])

# print(len(doc_embeddings))

### Заполнение векторной БД

In [8]:
# client.delete_collection("InternalDocs")

In [9]:
client = chromadb.Client()

collection = client.get_or_create_collection(name="InternalDocs")

collection.add(
    embeddings = doc_embeddings,
    documents = [doc_texts[0], doc_texts[1]],
    metadatas = [{"source": "register_account"},{"source": "vocation_request"}],
    ids = ["id1", "id2"]
)


### Поиск в БД документов релевантных запросу

In [10]:
query = "Как оформить заявку на 18 рабочих дней?"

query_embedding = json.loads(embeding_model.embed_query(doc))["embedding"]

relevant_doc = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)

relevant_doc_lgch =  [Document(page_content=relevant_doc['documents'][0][0], metadata=relevant_doc['metadatas'][0][0])]

### Формирование промпта

In [11]:
# page_content - is a key in langchain data type "Document", where to langchain expect find raw text
document_template = PromptTemplate(
    input_variables=["page_content"], #A list of the names of the variables.
    template="{page_content}" # The document template.
)

# The variable name where to put document in final prompt.
document_variable_name = "context"

# Template for prompt
template_override = """
    Представь что ты сотрудник компании MyApp. 
    Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
"""

# create prompt
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template_override
)

### Создание и выполнение цепочки 

In [12]:
llm_chain = LLMChain(llm=llm_model, prompt=prompt)

chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_template,
    document_variable_name=document_variable_name
)

In [13]:
result = chain.run(input_documents=relevant_doc_lgch, query=query)
print(result)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Согласно информации из текста, одна заявка не может быть оформлена более чем на 10 рабочих дней отпуска. Если сотруднику необходимо взять 18 рабочих дней отпуска, ему следует разделить этот период на несколько заявок, каждая из которых не превышает 10 рабочих дней. Для этого необходимо оформить несколько заявок на отпуск, каждая из которых соответствует требованиям, описанным в инструкции.

Также стоит учесть, что для оформления заявок на отпуск вне графика сотруднику необходимо обратиться к своему руководителю. В этом случае даты отпуска не будут включать в себя выходные дни, а количество дней отпуска может быть ограничено.
